# RetrievalQA Chain
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langsmith-cookbook/blob/main/hub-examples/retrieval-qa-chain/retrieval-qa.ipynb)

Developing a production-grade LLM application requires many refinements, but tracking multiple versions of prompts, models, and other components can be cumbersome. The [LangChain Hub](https://smith.langchain.com/hub) offers a centralized registry to manage and version your LLM artifacts efficiently. It even lets you interact with these artifacts directly in the browser to facilitate easier collaboration with non-technical team members.

[![Playground](https://github.com/langchain-ai/langsmith-cookbook/blob/main/hub-examples/retrieval-qa-chain/img/playground.png?raw=1)](https://smith.langchain.com/hub/rlm/rag-prompt/playground)

In its initial release (08/05/2023), the hub is limited to prompt management, but we plan to add support for other artifacts soon.

In this walkthrough, you will get started using the hub to manage prompts for a retrieval QA chain. You will go through the following steps:

1. Load prompt from Hub
2. Initialize Chain
3. Run Chain
4. Commit any new changes to the hub

## Prerequsites

#### a. Set up your LangSmith account

While you can access public prompts without an account, pushing new prompts to the hub requires a LangSmith account. Create your account at https://smith.langchain.com and log in.

Next, navigate to the [hub home](https://smith.langchain.com/hub). If you haven't already created a "handle", you will be prompted to do so. Your prompts and other artifacts will be saved within the namespace '<handle>/prompt-name', so choose one that you are comfortable sharing.

#### b. Configure environment

To use the hub, you'll want to use a recent version of LangChain and the `langchainhub` package. Install them with the following command:

In [1]:
%pip install -U langchain langchainhub --quiet

Finally, generate an API Key from your "personal" organization by navigating to the [LangSmith](https://smith.langchain.com) dashboard, and then set it in the cell below.

**Note:** Currently (08/04/2023), only API keys from your 'personal' organization are supported! If you see a '403' error at any point in this walkthrough, please confirm you've set a valid API key.

In [12]:
import os
from google.colab import userdata

# Update with your API URL if using a hosted instance of Langsmith.
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
# Update with your API URL if using a hosted instance of Langsmith.
os.environ["LANGCHAIN_HUB_API_URL"] = "https://api.hub.langchain.com"
os.environ["LANGCHAIN_HUB_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


## 1. Load prompt

Now it's time to load the prompt from the hub. We will use the `latest` version of [this retrieval QA prompt](https://smith.langchain.com/hub/rlm/rag-prompt) and later initialize the chain with it.

In [3]:
# RAG prompt
from langchain import hub

# Loads the latest version
prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

# To load a specific version, specify the version hash
# prompt = hub.pull("rlm/rag-prompt:50442af1")

## 2. Create the QA chain

Now that we've selected our prompt, initialize the chain.
 For this example, we will create a basic [RetrievalQA](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval_qa.base.RetrievalQA.html?highlight=retrievalqa#langchain.chains.retrieval_qa.base.RetrievalQA) over a vectorstore retriever.

Loading the data requires some amount of boilerplate, which we will run below.  While the specifics aren't important to this tutorial, you can learn more about Q&A in LangChain by visiting the [docs](https://python.langchain.com/docs/use_cases/question_answering/).

In [14]:
%pip install chromadb langchain_community tiktoken --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.3 MB/s eta 0:00:00


In [18]:
# Load docs
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings


loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

ModuleNotFoundError: No module named 'langchain_openai'

**Initialize the chain**. With the data added to the vectorstore, we can initialize the chain. We will
pass the prompt in via the `chain_type_kwargs` argument.

In [16]:
# RetrievalQA
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vectorstore.as_retriever(), chain_type_kwargs={"prompt": prompt}
)

## 3. Run Chain

Now that the chain is initialized, you can run it just like you would normally.

In [17]:
question = "What are the approaches to Task Decomposition?"
result = qa_chain({"query": question})
result["result"]

<ipython-input-17-563c10d15036>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


'The approaches to Task Decomposition include using LLM with simple prompting, task-specific instructions, and human inputs. Task decomposition involves breaking down large tasks into smaller subgoals for efficient handling of complex tasks and reflecting on past actions to improve future results. Challenges in long-term planning and task decomposition include planning over a lengthy history and effectively exploring the solution space.'

## Conclusion

In this tutorial, you learned how to use the [hub](https://smith.langchain.com/hub?page=1) to manage prompts for a retrieval QA chain. The hub is a centralized location to manage, version, and share your prompts (and later, other artifacts).

For more information, check out the [docs](https://docs.smith.langchain.com/hub/quickstart) or reach out to support@langchain.dev.